Importing library

In [1]:
import os
from astropy.io import fits
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import h5py
from keras.preprocessing.image import ImageDataGenerator

In [2]:
gpus = tf.config.list_physical_devices('GPU')

if gpus:
  # Restrict TensorFlow to only allocate 10GB of memory on the first GPU
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5000)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


Importing data

In [3]:
hf = h5py.File('../../data/HSC/HSC_v6/five_band_image120x120.hdf5', 'r')

OSError: Unable to open file (unable to open file: name = '../../data/HSC/HSC_v6/five_band_image120x120.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
x = hf["image"][0:5000]
y = hf["specz"][0:5000]

In [ ]:
x = np.transpose(x,(0,2,3,1))

In [ ]:
## check this
max_value = np.max(x)
max_value

In [ ]:
x_new = np.true_divide(x,max_value)

In [ ]:
hf.close()

Image Augmentation

In [ ]:
##datagen = ImageDataGenerator(rotation_range=90)
##it = datagen.flow(x_new, batch_size=1)

In [ ]:
x_train ,x_test,y_train,y_test = train_test_split(x_new,y,test_size=0.2)

In [ ]:
import keras
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU

In [ ]:
from sklearn.model_selection import train_test_split



model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='tanh',input_shape=(120,120,5)))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Conv2D(32, kernel_size=(2,2),activation='tanh'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Conv2D(32, kernel_size=(3,3),activation='relu'))
model.add(Conv2D(32, kernel_size=(2,2),activation='relu'))
model.add(Flatten())
model.add(Dense(500, activation="tanh"))
model.add(Dense(100, activation="tanh"))
model.add(Dense(1))



#model.compile(optimizer='Adam', loss="mse",metrics=[tf.keras.metrics.MeanAbsoluteError()])


model.summary()

In [ ]:
model.compile(optimizer='Adam', loss="mse",metrics=[tf.keras.metrics.MeanAbsoluteError()])

model.fit(x_train,y_train,epochs=300,shuffle = True,verbose=1)


In [ ]:
y_test_predict = model.predict(x_test)
y_test_prediction = []

for i in range(len(y_test)):
    y_test_prediction.append(y_test_predict[i][0])

y_test_prediction = np.asarray(y_test_prediction)

y_test_ = []

for i in range(len(y_test)):
    y_test_.append(y_test[i][0])

y_test_ = np.asarray(y_test_)

In [ ]:
from photoz_utils import *
y_test_new = pd.Series(y_test_)
y_test_predict_new = pd.Series(y_test_prediction)
get_point_metrics(y_test_predict_new,y_test_new)

In [ ]:
binned_metrics_test = get_point_metrics(y_test_predict_new,y_test_new,binned = True)
plot_point_metrics(binned_metrics_test)

In [ ]:
y_train_predict = model.predict(x_train)
y_train_prediction = []

for i in range(len(y_train)):
    y_train_prediction.append(y_train_predict[i][0])

y_train_prediction = np.asarray(y_train_prediction)

y_train_ = []

for i in range(len(y_train)):
    y_train_.append(y_train[i][0])

y_train_ = np.asarray(y_train_)

In [ ]:
from photoz_utils import *
y_train_new = pd.Series(y_train_)
y_train_predict_new = pd.Series(y_train_prediction)
get_point_metrics(y_train_predict_new,y_train_new)

In [ ]:
binned_metrics_train = get_point_metrics(y_train_predict_new,y_train_new,binned = True)
plot_point_metrics(binned_metrics_train)

In [ ]:
from scipy.stats import gaussian_kde
xy = np.vstack([y_test_,y_test_prediction])
z = gaussian_kde(xy)(xy)

In [ ]:
import matplotlib.cm as cm
import matplotlib
fig, ax = plt.subplots()
ax_ = ax.scatter(y_test, y_test_prediction, c=z,s = 1,edgecolor=None, norm=matplotlib.colors.LogNorm())
plt.colorbar(ax_)
plt.xlabel('spectroscopic redshift')
plt.ylabel('photo z')
plt.plot([.18,1.6*2.4],[0,1.2*2.4], color='black')
plt.plot([0, 1.6 * 2.4], [.15, 2 * 2.4],color = 'black')